In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import yaml

#Criar sessão Spark
spark = SparkSession.builder \
    .appName('RNP - E-Commerce Case - Victor Hugo') \
    .config('spark.sql.legacy.pathOptionBehavior.enabled', True) \
    .config('spark.databricks.delta.formatCheck.enabled', False)  \
    .getOrCreate()

# Configurar as opções de conexão ao banco de dados
credenciais = yaml.load(open('./credenciais.yml'))
propriedades = {
    "user": credenciais['database']['username'],
    "password": credenciais['database']['password'],
    "driver": credenciais['database']['driver'],
    "url": credenciais['database']['url']
}

jdbc_url = propriedades["url"]

In [ ]:
# QUESTÃO 3 - No Notebook crie lógica para salvar cada tabela como parqueT

# Configurações de conexão com o banco de dados
database_url = propriedades['url']

# Obtendo a lista de todas as tabelas do banco de dados
query = "(SELECT table_name FROM information_schema.tables WHERE table_schema = 'public') AS tables"
ecommerce_tables = spark.read.jdbc(url=database_url, table=query, properties=propriedades)

# Excluindo as tabelas que você deseja evitar
tables_to_exclude = ["pg_stat_statements", "pg_buffercache"]
filtered_tables_df = ecommerce_tables.filter(~ecommerce_tables.table_name.isin(tables_to_exclude))

# Coletando a lista final de tabelas a serem salvas como Parquet
tabelas = filtered_tables_df.select("table_name").rdd.flatMap(lambda x: x).collect()

# Loop através das tabelas e salvando como Parquet
for tabela in tabelas:
    table_df = spark.read \
        .jdbc(url=database_url, table=tabela, properties=propriedades)
    
    # Salvando a tabela como Parquet
    table_df.write.parquet(f"/dbfs/Users/victorhacmo@gmail.com/parquets/{tabela}.parquet")

# Encerrando a sessão Spark
spark.stop()


In [ ]:
#FERRAMENTA AUXILIAR PARA AS RESPOSTAS DA QUESTÃO 5

# Esta função utiliza o Spark para consultar dados de e-commerce de uma fonte JDBC especificada pela URL e tabela fornecidas. Os resultados são armazenados em um arquivo Delta no caminho especificado, permitindo a substituição do esquema anterior. Em caso de erro, a função exibe uma mensagem de erro detalhada.

def query_ecommerce_data (url, table, properties):
    try:
        df = spark.read.jdbc(url,
                            table,
                            properties=propriedades)
        file_path = f'/dbfs/Users/victorhacmo@gmail.com/dados_delta'
        df.write.format('delta').mode('overwrite').option("overwriteSchema", "true").save(f'{file_path}')
        return df
    except Exception as e:
        print(f'Erro: {e}')

In [ ]:
# QUESTÃO 5.1 - Qual país possui a maior quantidade de itens cancelados?

query_Q5_1 = '''
    (SELECT DISTINCT
        C.COUNTRY AS PAIS,
        SUM(DOS.QUANTITY_ORDERED) AS TOTAL_ITENS_CANCELADOS
    FROM 
        PUBLIC.CUSTOMERS C 
    LEFT JOIN 
        PUBLIC.ORDERS OS ON C.CUSTOMER_NUMBER = OS.CUSTOMER_NUMBER
    JOIN 
        ORDERDETAILS DOS ON DOS.ORDER_NUMBER = OS.ORDER_NUMBER
    WHERE 
        OS.STATUS = 'Cancelled'
    GROUP BY 
        C.COUNTRY
    ORDER BY 
        TOTAL_ITENS_CANCELADOS DESC
    LIMIT 1) AS subquery
'''

df_qtd_cancelados = query_ecommerce_data(jdbc_url, query_Q5_1, propriedades)

In [ ]:
df_qtd_cancelados.show()

+-----+----------------------+
| pais|total_itens_cancelados|
+-----+----------------------+
|Spain|                   605|
+-----+----------------------+



In [ ]:
# QUESTÃO 5.2 - Qual o faturamento da linha de produto mais vendido, considere os itens com status 'Shipped', cujo o pedido foi realizado no ano de 2005?

query_Q5_2 = '''
(SELECT 
    PD.PRODUCT_LINE,
        SUM(P.AMOUNT) AS FATURAMENTO
FROM 
    PRODUCTS PD
LEFT JOIN 
    ORDERDETAILS OD
ON 
    OD.PRODUCT_CODE  = PD.PRODUCT_CODE 
LEFT JOIN 
    ORDERS O
ON 
    O.ORDER_NUMBER  = OD.ORDER_NUMBER 
LEFT JOIN 
    PAYMENTS P 
ON 
    P.CUSTOMER_NUMBER = O.CUSTOMER_NUMBER 
WHERE 
    O.STATUS = 'Shipped'
AND 
    EXTRACT(YEAR FROM O.ORDER_DATE) = 2005
GROUP BY 
    PD.PRODUCT_LINE
ORDER BY 
    FATURAMENTO DESC
LIMIT 1) As subquery
'''

df_produto_2005 = query_ecommerce_data(jdbc_url, query_Q5_2, propriedades)

In [ ]:
df_produto_2005.show()

+------------+--------------------+
|product_line|         faturamento|
+------------+--------------------+
|Classic Cars|47574393.22000000...|
+------------+--------------------+



In [ ]:
# QUESTÃO 5.3 - Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail deve estar mascarado.

query_Q5_3 = '''
(SELECT
        E.FIRST_NAME AS NOME,
        E.LAST_NAME AS SOBRENOME,
        CONCAT(REPEAT('*', POSITION('@' IN E.EMAIL) - 1), SUBSTRING(E.EMAIL, POSITION('@' IN E.EMAIL))) AS EMAIL
    FROM 
        EMPLOYEES E 
    LEFT JOIN 
        OFFICES O 
    ON 
        O.OFFICE_CODE = E.OFFICE_CODE 
    WHERE 
        O.COUNTRY = 'Japan'
) As subquery
'''

df_vendedores_japao = query_ecommerce_data(jdbc_url, query_Q5_3, propriedades)

In [ ]:
df_vendedores_japao.show()

+-------+---------+--------------------+
|   nome|sobrenome|               email|
+-------+---------+--------------------+
|   Mami|    Nishi|******@classicmod...|
|Yoshimi|     Kato|*****@classicmode...|
+-------+---------+--------------------+

